## Adding Tabular data to Azure Blob Storage and Accessing Sentinel-2 L2A data with the Planetary Computer STAC API

## Before beginning this notebook and for later work on this project, please get the credentials for the:

- ACCOUNT_NAME 
    - See gif below {in this example account is `fluviusdata`}
- BLOB_KEY 
    - See gif below {in this example key is found under `Key`}
- CONNECTION_STRING 
    - See gif below {in this example key is found under `Connection string`}
- PLANETARY_COMPUTER_SAS_TOKEN
    - Note that connection string is not necessary, and the Planetary Computer SAS Token can be acquired by applying for Planetary Computer access [here](https://planetarycomputer.microsoft.com/account/request)

![credentials](https://fluviusdata.blob.core.windows.net/example/credential_demo.gif) 

## At this point we will want to move data that we have locally to the container in Azure Blob Storage. Here we demonstrate uploading data to Azure Blob Storage via the portal GUI ![example](https://fluviusdata.blob.core.windows.net/example/data_upload_demo.gif)

## Now that data is available to us in the Cloud, we can access them here in this notebook utilizing the credentials we have in the `credentials` file.

In [1]:
#import all the libraries we will need 

import os
import fsspec 
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython.display import display, Image
from pystac_client import Client
import planetary_computer as pc

#import the fluvius library
import sys #need to add the current directory to import 
sys.path.append('/content')
from src.fluvius import WaterData, WaterStation
from src.utils import generate_map


In [2]:
# reads the credential file
with open('/content/credentials') as credentials:
    f = credentials.readlines() #gets the individual lines
    
# now assign those values to os.environ as accessible variables
for var in f:
    key, value = var.split(' = ') 
    #be sure the ' = 'has a space before and after in the credentials file
    os.environ[key] = value.rstrip('\n')

# then we will store our keys into a variable called storage_options

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY'],
                 'connection_string': os.environ['CONNECTION_STRING']}

fs = fsspec.filesystem('az',\
                       account_name=storage_options['account_name'],\
                       account_key=storage_options['account_key'])   


## At this point we can use our credentials to open the file we just opened

In [3]:
df = pd.read_csv('az://example/usgs_station_metadata_example.csv',\
                storage_options=storage_options)

## Note this is the same as reading a file with just using pandas, but the addtional assigning of the `storage_option` parameter.

In [4]:
df

,site_no,site_name,Latitude,Longitude,geometry
0,7182250,"Cottonwood River at Plymouth, KS",38.3975,-96.3561,POINT (-96.3561 38.3975)
1,7182390,"Neosho River near Neosho Rapids, KS",38.3680,-96.0000,POINT (-96 38.368)
2,9326500,"Ferron Creek (Upper Station) near Ferron, UT",39.1041,-111.2170,POINT (-111.217 39.1041)
3,9327000,Ferron Creek Below Millsite Res & Divs Near Fe...,39.0953,-111.1790,POINT (-111.179 39.0953)
4,1673000,"Pamunkey River Near Hanover, VA",37.7676,-77.3322,POINT (-77.3322 37.7676)
5,6805500,"Platte River at Louisville, NE",41.0152,-96.1577,POINT (-96.15770000000001 41.0152)
6,11455146,Liberty Cut at Little Holland Tract near Court...,38.3288,-121.6680,POINT (-121.668 38.3288)
7,6795500,"Shell Creek near Columbus, NE",41.5261,-97.2817,POINT (-97.2817 41.5261)
8,1649190,"Paint Branch Near College Park, MD",39.0331,-76.9643,POINT (-76.96429999999999 39.0331)
9,1478245,"White Clay Creek near Strickersville, PA",39.7475,-75.7708,POINT (-75.77079999999999 39.7475)


## With this current data, we have a `Latitude` and `Longitude` column. Here we can use `geopandas` to create a `POINT` data object and then visualize it with `folium`. We built helper function `generate_map` to do this for us. 

In [10]:
m = generate_map(df, lat_colname='Latitude', lon_colname='Longitude')
m

## Alternatively we can use the `WaterData` class in the `fluvius` library to get our data that has already been processed  

In [7]:
#declare the data source we are using
#choices are ['itv', 'ana', 'usgs', 'usgsi']
data_source = 'itv'
container = f'{data_source}-data'
ds = WaterData(data_source, container, storage_options)
ds.get_source_df()

## Now we want to build a 'chip' around our points. This chip will also serve as the area of interest (AOI) that we will submit to the STAC API to query for Sentinel-2 data. Again we use a help function called `apply_buffer_to_points` in the WaterData Class

In [8]:
buffer_distance = 500
ds.apply_buffer_to_points(buffer_distance)

## Let's plot our map here using the `generate_map` and `plot_map` functions 

In [9]:
ds.generate_map()
ds.plot_map

## Next we will demonstrate creating a time buffer to create a time of interest 'TOI'


Show how to format these AOI and TOI to something readable with PySTAC 

Demonstrate saving tabular data using pandas API for connecting into Azure Blob Storage 

Show how we use these two parameters to query the Sentinel-2 catalog 

Explain the items returned by the STAC API and how to read meta data 

Explain how to filter by various metadata such as cloud cover 

Explain how to get assets matching the bands of interest 

Demonstrate how to get the open those hyperlinks with rasterio and then clip to the AOI  

Demonstrate saving this data into a folder in Azure Blob Storage using Azure Container Client as a geotiff

In [1]:
#END